---
# Diagnosis of mechanical rotor faults in drones using functional gaussian mixture classifier
B. Bartoszewski,
K. Jarzyna
---

---

### Problem formulation [0-4 pts]: Bartosz
  - is the problem clearly stated [1 pt]
  - what is the point of creating model, are potential use cases defined [1 pt]
  - where do data comes from, what does it containt [1 pt]
  - is preprocessing step clearly described [1 pt]

---
### Data preprocessing   [0-2 pts]: Bartosz
- reasoning and types of actions taken on the dataset have been described [1 pt]
- is preprocessing step clearly described [1 pt]


---
### Model [0-4 pts]: Bartosz 2,3 
- are models sufficiently described (what are formulas, what are parameters, what data are required ) [1 pt] 
- are difference between two models explained [1 pt]  
- is the difference in the models justified (e.g. does adding aditional parameter makes sense? ) [1 pt]  
- are two different models specified [1 pt]

---
### Priors [0-4 pts]: Kacper
- Have prior predictive checks been done for parameters (are parameters simulated from priors make sense) [1 pt]
- How prior parameters were selected [1 pt]
- Have prior predictive checks been done for measurements (are measurements simulated from priors make sense) [1 pt]
- Is it explained why particular priors for parameters were selected [1 pt]


---
### Posterior analysis (model 1) [0-4 pts]: KAcper
- have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt] 
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)
- are the samples from posterior predictive distribution analyzed [1 pt] 

---
### Posterior analysis (model 2) [0-4 pts]:
- have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt]
- are the samples from posterior predictive distribution analyzed [1 pt]
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)

---
### Model comparison [0-4 pts]: Bartosz, Kacper
- Have models been compared using information criteria [1 pt]
- Have result for WAIC been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Have result for PSIS-LOO been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Was the model comparison discussed? Do authors agree with information criteria? Why in your opinion one model better than another [1 pt]
